In [1]:
!pip install package_name

  Preparing metadata (setup.py) ... done
  Created wheel for package_name: filename=package_name-0.1-py3-none-any.whl size=1229 sha256=0eb34e2f93ab14b4f20c8b1f60829ec36c61b2a8b9b7fd1d5aa9271b398f5325
  Stored in directory: /root/.cache/pip/wheels/a3/36/c4/98d928f30290fb88555f848f73093f02b67c984a45c56c3e97
Successfully built package_name


In [5]:
!pip install pydub==0.25.1

In [11]:
!pip install pydub


In [13]:
import os
import subprocess
import requests
import pandas as pd
from pydub import AudioSegment
from pydub.exceptions import CouldntDecodeError

# Load dataset with audio URLs
file_path = '/content/audio_dataset.csv'
audio_data = pd.read_csv(file_path)

# Print the column names to verify
print(audio_data.columns)

# Strip extra spaces from column names
audio_data.columns = audio_data.columns.str.strip()

# Create directory to store downloaded audio files
output_dir = "downloaded_audio"
os.makedirs(output_dir, exist_ok=True)

# Function to download audio from a URL
def download_audio(url, output_dir, audio_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            file_path = os.path.join(output_dir, f"{audio_name}.mp3")
            with open(file_path, "wb") as f:
                f.write(response.content)
            return file_path
        else:
            print(f"Failed to download {url}")
            return None
    except Exception as e:
        print(f"Error while downloading {url}: {e}")
        return None

# Function to convert MP3 to WAV using ffmpeg
def convert_to_wav(mp3_file):
    wav_file = mp3_file.replace(".mp3", ".wav")

    # Run ffmpeg command and capture any errors
    conversion_command = f"ffmpeg -i {mp3_file} {wav_file}"
    process = subprocess.run(conversion_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Log the output of the conversion process
    if process.returncode != 0:
        print(f"FFmpeg error while converting {mp3_file}: {process.stderr.decode('utf-8')}")
        return None
    elif os.path.exists(wav_file):
        return wav_file
    else:
        print(f"Conversion failed for {mp3_file}.")
        return None

# Download the first 20 audio files
audio_files = []
for i, row in audio_data.head(20).iterrows():
    audio_name = row['audio name'].strip()  # Ensure correct column name
    file_path = download_audio(row['file path'], output_dir, audio_name)

    # Check if file was downloaded successfully
    if file_path and os.path.exists(file_path):
        print(f"Downloaded {file_path}")

        # Convert to WAV format
        wav_file_path = convert_to_wav(file_path)
        if wav_file_path:
            audio_files.append(wav_file_path)
    else:
        print(f"Skipping {audio_name} due to download issues.")

# Merge the audio files into one
merged_audio = AudioSegment.empty()
for audio_file in audio_files:
    try:
        audio_segment = AudioSegment.from_wav(audio_file)
        merged_audio += audio_segment
    except CouldntDecodeError:
        print(f"Failed to decode {audio_file}. Skipping...")
        continue
    except FileNotFoundError:
        print(f"File not found: {audio_file}")
        continue

# Save the merged audio
merged_audio.export(os.path.join(output_dir, "merged_audio.wav"), format="wav")

# Create CSV file with details
audio_details = {
    "audio_name": [os.path.basename(f) for f in audio_files],
    "file_path": audio_files,
    "duration_seconds": [AudioSegment.from_wav(f).duration_seconds for f in audio_files]
}
audio_df = pd.DataFrame(audio_details)
audio_df.to_csv("merged_audio_details.csv", index=False)

print("Audio merged and CSV file created.")


Index(['audio name ', 'file path', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10'],
      dtype='object')
Downloaded downloaded_audio/audio 1.mp3
FFmpeg error while converting downloaded_audio/audio 1.mp3: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --en